In [ ]:
import pandas as pd
import sys
import torch
from transformers import GPT2Config, GPT2Tokenizer, GPT2LMHeadModel, GPT2Model
from attention_analysis import *
from bertviz import head_view, model_view

In [2]:
model = GPT2LMHeadModel.from_pretrained('gpt2', output_attentions = True)
tokenizer = GPT2Tokenizer.from_pretrained('gpt2')
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

In [3]:
def find_all_elem_index(l,e):
    retVal = []
    if e in l:
        for idx in range(len(l)):
            if l[idx]==e:
                retVal.append(idx)
    else:
        for idx in range(len(l)):
            frag = l[idx]
            if e.startswith(frag):
                string=frag
                for idx_2 in range(idx+1, len(l)):
                    frag_2 = l[idx_2]
                    string += frag_2
                    if string == e:
                        retVal.append(idx_2)
                        break
    return retVal

In [4]:
def get_surprisal(sent, roi_loc):
    indexed_tokens = tokenizer.encode(sent)
    tokens = [tokenizer.decode(i).strip() for i in indexed_tokens]

    prefix_index = indexed_tokens[:roi_loc]
    #print(tokens[:roi_loc])
    prefix_tensor = torch.tensor([prefix_index])
    prefix_tensor = prefix_tensor.to(device)
    model.to(device)
    with torch.no_grad():
        output= model(prefix_tensor)
        predictions = torch.nn.functional.softmax(output[0],-1)
        result = predictions[0,-1,:]
        score_current_w = result[indexed_tokens[roi_loc]]
        surprisal_current_w = -1*torch.log2(score_current_w)
        surprisal_current_w = surprisal_current_w.numpy()
    return float(surprisal_current_w)

## Wagers_exp3

In [5]:
W2009_3 = pd.read_csv('materials/WagersExp3.csv')
W2009_3 

,id,distractor,subject,verb,interference,grammaticality,sentence,crit,target,attractor
0,1,sing,sing,sing,int,gram,The runner who the driver sees during the comm...,sees,driver,runner
1,1,pl,sing,sing,unint,gram,The runners who the driver sees during the com...,sees,driver,runners
2,1,sing,sing,pl,unint,ungram,The runner who the driver see during the commu...,see,driver,runner
3,1,pl,sing,pl,int,ungram,The runners who the driver see during the comm...,see,driver,runners
4,1,sing,pl,pl,unint,gram,The runner who the drivers see during the comm...,see,drivers,runner
...,...,...,...,...,...,...,...,...,...,...
379,48,pl,sing,pl,int,ungram,The programmers who the manager oversee at the...,oversee,manager,programmers
380,48,sing,pl,pl,unint,gram,The programmer who the managers oversee at the...,oversee,managers,programmer
381,48,pl,pl,pl,int,gram,The programmers who the managers oversee at th...,oversee,managers,programmers
382,48,sing,pl,sing,int,ungram,The programmer who the managers oversees at th...,oversees,managers,programmer


In [18]:
df= pd.DataFrame()
W2009_3_id = np.unique(W2009_3['id'])
W2009_3_subject = np.unique(W2009_3['subject'])
W2009_3_distractor = np.unique(W2009_3['distractor'])
W2009_3_verb = np.unique(W2009_3['verb'])

for i in W2009_3_id:
    this_set =W2009_3.loc[W2009_3['id']==i]
    for subject in W2009_3_subject:
        for distractor in W2009_3_distractor:
            for verb in W2009_3_verb:
                surprisal, attn_to_target1, entropy1 = None,None,None
                this_sent = this_set.loc[this_set['subject']==subject].loc[this_set['distractor']==distractor].loc[this_set['verb']==verb]
                sent=this_sent['sentence'].values[0].strip()
                crit = this_sent['crit'].values[0].strip()
                target = this_sent['target'].values[0].strip()
                attractor = this_sent['attractor'].values[0].strip()
                attn_sent = attention(sent,model,tokenizer)

                if target in attn_sent.tokens and crit in attn_sent.tokens:
                    crit_loc=find_all_elem_index(attn_sent.tokens,crit)[-1]
                    target_loc = find_all_elem_index(attn_sent.tokens,target)[-1]
                    attn_to_target1 = attn_sent.attn_data['all']['attn'][4][3][crit_loc][target_loc] 
                    
                if crit in attn_sent.tokens:
                    crit_loc=find_all_elem_index(attn_sent.tokens,crit)[-1]
                    attn_to_target = None    
                    surprisal = get_surprisal(sent, crit_loc)
                    entropy1 = entropy(attn_sent.attn_data['all']['attn'][4][3][crit_loc], base=2)

   
                df = df.append({'id':i,
                                   "subject":subject,
                                    "distractor":distractor,
                                    "verb":verb,
                                    "grammaticality":this_sent['grammaticality'].values[0],
                                    "interference":this_sent['interference'].values[0],
                                   "sentence":sent,
                                   "crit":crit,
                                   "surprisal":surprisal,
                                   "entropy": entropy1,
                                    
                                   "attn_to_target":attn_to_target1,                              
                              },ignore_index=True)


In [19]:
df.to_csv('results/W2009_exp3.csv')


In [20]:
W2009_4_5 = pd.read_csv('materials/WagersExp4_5.csv') 

In [21]:
df= pd.DataFrame()
W2009_4_5_id = np.unique(W2009_4_5['id'])
W2009_4_5_distractor = np.unique(W2009_4_5['distractor'])
W2009_4_5_verb = np.unique(W2009_4_5['verb'])

for i in W2009_4_5_id:
    this_set =W2009_4_5.loc[W2009_4_5['id']==i]
    for distractor in W2009_4_5_distractor:
        for verb in W2009_4_5_verb:
            surprisal, attn_to_target1, entropy1 = None,None,None
            this_sent = this_set.loc[this_set['distractor']==distractor].loc[this_set['verb']==verb]
            sent=this_sent['sentence'].values[0].strip()
            crit = this_sent['crit'].values[0].strip()
            target = this_sent['target'].values[0].strip()
            attractor = this_sent['attractor'].values[0].strip()
            attn_sent = attention(sent,model,tokenizer)

            if target in attn_sent.tokens and crit in attn_sent.tokens:
                crit_loc=find_all_elem_index(attn_sent.tokens,crit)[-1]
                target_loc = find_all_elem_index(attn_sent.tokens,target)[-1]
                attn_to_target1 = attn_sent.attn_data['all']['attn'][4][3][crit_loc][target_loc] 
                    
            if crit in attn_sent.tokens:
                crit_loc=find_all_elem_index(attn_sent.tokens,crit)[-1]
                attn_to_target = None    
                surprisal = get_surprisal(sent, crit_loc)
                entropy1 = entropy(attn_sent.attn_data['all']['attn'][4][3][crit_loc], base=2)

            df = df.append({'id':i,
                                   "subject":subject,
                                    "distractor":distractor,
                                    "verb":verb,
                                    "grammaticality":this_sent['grammaticality'].values[0],
                                    "interference":this_sent['interference'].values[0],
                                   "sentence":sent,
                                   "crit":crit,
                                   "surprisal":surprisal,
                                   "entropy": entropy1,
                                   "attn_to_target":attn_to_target1},ignore_index=True)
                

In [22]:
df.to_csv('results/W2009_exp45.csv')

In [23]:
D2013_A = pd.read_csv('materials/Dillon2013_agreement.csv')

In [24]:
df= pd.DataFrame()
D2013_A_id = np.unique(D2013_A['id'])
D2013_A_subject = np.unique(D2013_A['subject'])
D2013_A_distractor = np.unique(D2013_A['distractor'])
D2013_A_grammaticality = np.unique(D2013_A['grammaticality'])

for i in D2013_A_id:
    this_set =D2013_A.loc[D2013_A['id']==i]
    for subject in D2013_A_subject:
        for distractor in D2013_A_distractor:
            for grammaticality in D2013_A_grammaticality:
                surprisal, attn_to_target1, entropy1 = None,None,None
                this_sent = this_set.loc[this_set['subject']==subject].loc[this_set['distractor']==distractor].loc[this_set['grammaticality']==grammaticality]
                sent=this_sent['sentence'].values[0].strip()
                crit = this_sent['crit'].values[0].strip()
                target = this_sent['target'].values[0].strip()
                attractor = this_sent['attractor'].values[0].strip()
                attn_sent = attention(sent,model,tokenizer)


                if target in attn_sent.tokens and crit in attn_sent.tokens:
                    crit_loc=find_all_elem_index(attn_sent.tokens,crit)[-1]
                    target_loc = find_all_elem_index(attn_sent.tokens,target)[-1]
                    attn_to_target1 = attn_sent.attn_data['all']['attn'][4][3][crit_loc][target_loc] 
                    
                if crit in attn_sent.tokens:
                    crit_loc=find_all_elem_index(attn_sent.tokens,crit)[-1]
                    attn_to_target = None    
                    surprisal = get_surprisal(sent, crit_loc)
                    entropy1 = entropy(attn_sent.attn_data['all']['attn'][4][3][crit_loc], base=2)
                
                df = df.append({'id':i,
                                   "subject":subject,
                                    "distractor":distractor,
                                    "verb":verb,
                                    "grammaticality":this_sent['grammaticality'].values[0],
                                    "interference":this_sent['interference'].values[0],
                                   "sentence":sent,
                                   "crit":crit,
                                   "surprisal":float(surprisal),
                                   "entropy": entropy1,
                                   "attn_to_target":attn_to_target1,                              
                              },ignore_index=True)


In [25]:
df.to_csv('results/Dillon2013_agreement.csv')

## Dillon - Reflexive

In [26]:
D2013_R = pd.read_csv('materials/Dillon2013_reflexive.csv')

In [27]:
df= pd.DataFrame()
D2013_R_id = np.unique(D2013_R['id'])
D2013_R_subject = np.unique(D2013_R['subject'])
D2013_R_distractor = np.unique(D2013_R['distractor'])
D2013_R_grammaticality = np.unique(D2013_R['grammaticality'])

for i in D2013_R_id:
    this_set =D2013_R.loc[D2013_R['id']==i]
    for subject in D2013_R_subject:
        for distractor in D2013_R_distractor:
            for grammaticality in D2013_R_grammaticality:
                surprisal, attn_to_target1, entropy1 = None,None,None
                this_sent = this_set.loc[this_set['subject']==subject].loc[this_set['distractor']==distractor].loc[this_set['grammaticality']==grammaticality]
                sent=this_sent['sentence'].values[0].strip()
                crit = this_sent['crit'].values[0].strip()
                target = this_sent['target'].values[0].strip()
                attractor = this_sent['attractor'].values[0].strip()
                attn_sent = attention(sent,model,tokenizer)
                if target in attn_sent.tokens and crit in attn_sent.tokens:
                    crit_loc=find_all_elem_index(attn_sent.tokens,crit)[-1]
                    target_loc = find_all_elem_index(attn_sent.tokens,target)[-1]
                    attn_to_target1 = attn_sent.attn_data['all']['attn'][4][3][crit_loc][target_loc] 
                    
                if crit in attn_sent.tokens:
                    crit_loc=find_all_elem_index(attn_sent.tokens,crit)[-1]
                    attn_to_target = None    
                    surprisal = get_surprisal(sent, crit_loc)
                    entropy1 = entropy(attn_sent.attn_data['all']['attn'][4][3][crit_loc], base=2)
               
                df = df.append({'id':i,
                                   "subject":subject,
                                    "distractor":distractor,
                                    "verb":verb,
                                    "grammaticality":this_sent['grammaticality'].values[0],
                                    "interference":this_sent['interference'].values[0],
                                   "sentence":sent,
                                   "crit":crit,
                                   "surprisal":float(surprisal),
                                   "entropy": entropy1,
                                   "attn_to_target":attn_to_target1,                              
                              },ignore_index=True)


In [28]:
df.to_csv('results/Dillon2013_reflexive.csv')